# Arrays

## Arrays vs Linked List

Arrays and linked lists are both data structures used to store collections of elements, but they have distinct characteristics:

1. Arrays:

 - Fixed Size: The size of an array is defined at creation and cannot be changed.
 - Contiguous Memory: Elements are stored in contiguous memory locations, allowing for fast access (`O(1)` time complexity) using an index.
 - Easy to Use: Simple to implement and access elements.
 - Insertion/Deletion: Adding or removing elements can be costly (`O(n)` time complexity) as it may require shifting elements.

2. Linked Lists:

 - Dynamic Size: Can grow or shrink in size as needed, allowing for more flexible memory usage.
 - Non-contiguous Memory: Elements (nodes) are stored in separate memory locations, each containing a reference (pointer) to the next node.
 - Complexity: More complex to implement and manage due to pointers.
 - Insertion/Deletion: Easier and more efficient (`O(1)` time complexity) for adding or removing elements, especially at the beginning or end of the list.

In summary, arrays provide fast access and are simpler to use, while linked lists offer flexibility in size and efficient insertions/deletions.



## ArrayList

An ArrayList is a resizable array implementation. It allows for dynamic storage of elements, meaning it can grow or shrink in size as elements are added or removed. ArrayLists provide fast access to elements via indexing and are useful for scenarios where frequent read operations are needed. However, adding or removing elements, especially in the middle of the list, can be slower compared to other data structures like LinkedLists due to the need for shifting elements.


- Bad performance with `enqueue()/prepend()` and `deque()/removeAt(0)` - `O(N)`
- Great performance with `push()/append()` and `pop()` - `O(1)`

In [24]:
from typing import TypeVar, Optional, List

T = TypeVar("T") 

class ArrayList[T]:
    length: int = 0
    capacity: int = 0
    _elements: List[T] = None

    def __init__(self, capacity: int = 2):
        # simulates memory region
        self._elements = [None,] * capacity
        self.capacity = capacity

    def prepend(self, element: T):
        if self.is_full():
            self._expand_array()
        for i in range(self.length, 0, -1):
            self._elements[i] = self._elements[i-1]
        
        self._elements[0] = element
        self.length += 1

    def is_full(self):
        return self.length == self.capacity

    def insertAt(self, element: T, index: int):
        if self.is_full():
            self._expand_array()
        for i in range(self.length, index-1, -1):
            self._elements[i] = self._elements[i-1]

        self._elements[index] = element
        self.length += 1

    def _expand_array(self):
        current = self._elements
        new_capacity = self.capacity * 2
        self._elements = [None] * new_capacity
        for i in range(self.capacity):
            self._elements[i] = current[i]

        self.capacity = new_capacity

    def append(self, element: T):
        if self.is_full():
            self._expand_array()
        self._elements[self.length] = element
        self.length += 1

    def remove(self,  element: T):
        return self.removeAt(self.indexOf(element))
    
    def get(self, index) -> Optional[T]:
        if index < 0 or index >= self.length:
            return
        return self._elements[index]
    
    def indexOf(self, element: T):
        for i in range(0, self.length):
            if self._elements[i] == element:
                return i
        return -1
    
    def removeAt(self, index: int) -> Optional[T]:
        if index < 0 or index >= self.length:
            return None
        value = self._elements[index]
        for i in range(index, self.length):
            self._elements[i] = self._elements[i+1]
        self.length -= 1
        return value
    
    def pop(self):
        value = self._elements[self.length-1]
        self._elements[self.length-1] = None
        self.length -= 1
        return value


    def __len__(self) -> int:
        return self.length

    def __str__(self) -> str:
        return "[" + ", ".join(repr(e) for e in self._elements) + "]"
    

items = ArrayList()

items.append(5)
items.append(7)
items.append(9)
assert items.get(2) == 9
assert items.removeAt(1) == 7
assert len(items) == 2

items.append(11)

assert items.removeAt(1) == 9
assert items.removeAt(9) is None
assert items.removeAt(0) == 5
assert items.removeAt(0) == 11
assert len(items) == 0

items.prepend(5)
items.prepend(7)
items.prepend(9)

assert items.get(2) == 5
assert items.get(0) == 9
assert items.remove(9) == 9
assert len(items) == 2
assert items.get(0) == 7

items.insertAt(3, 1)

assert len(items) == 3
assert items.get(0) == 7
assert items.get(1) == 3
assert items.get(2) == 5

assert items.indexOf(3) == 1

assert items.pop() == 5
assert len(items) == 2
assert items.pop() == 3
assert items.pop() == 7


## Ring Buffer

A ring buffer, also known as a circular buffer, is a fixed-size data structure that uses a single, contiguous block of memory to store a collection of elements in a circular manner. When the buffer reaches its capacity, new data overwrites the oldest data, making it efficient for scenarios where data is continuously produced and consumed, such as in streaming applications or real-time data processing. The ring buffer maintains two pointers (or indices) to track the start and end of the data, allowing for efficient insertion and removal of elements without the need for shifting data.

In [105]:
from typing import TypeVar, Optional, List

T = TypeVar("T") 

class RingBuffer[T]():
    capacity: int = 0
    length: int = 0
    head: int = 0
    tail: int = 0
    buffer: List[T] = None

    def __init__(self, capacity: int = 3):
        self.capacity = capacity
        self.buffer = [None] * capacity

    def is_full(self):
        return self.length == self.capacity
    
    def push(self, element: T):
        if self.is_full():
            raise NotImplementedError("Extending capacity has not been implemented.")
        self.buffer[self.tail] = element
        self.tail = (self.tail + 1) % self.capacity
        self.length += 1


    def pop(self) -> Optional[T]:
        if len(self) == 0:
            return
        index = (self.tail - 1) % self.capacity
        element = self.buffer[index]
        self.buffer[index] = None
        self.tail = index
        self.length -= 1
        return element

    def get(self, index: int) -> Optional[T]:
        if index >= self.length:
            return
        
        return self.buffer[(self.head + index) % self.capacity]
    
    def deque(self):
        element = self.buffer[self.head]
        self.buffer[self.head] = None
        self.head = (self.head + 1) % self.capacity
        self.length -= 1
        return element


    def __len__(self):
        return self.length
    
    def __str__(self) -> str:
        buffer = "buffer: [" + ", ".join(repr(e) for e in self.buffer) + "]"
        ring = "ring: [" + ", ".join(repr(self.buffer[i % self.capacity]) for i in range(self.head, self.head + self.length)) + "]"

        return f"<RingBuffer head={self.head} tail={self.tail} len={len(self)} " + ring + " " + buffer + " >"



buffer = RingBuffer()

buffer.push(5)

print(buffer)

assert buffer.pop() == 5
assert buffer.pop() is None

buffer.push(42)
buffer.push(9)

print(buffer)

assert buffer.pop() == 9
assert buffer.pop() == 42
assert buffer.pop() is None

buffer.push(42)
buffer.push(9)
buffer.push(12)

assert buffer.get(2) == 12
assert buffer.get(1) == 9
assert buffer.get(0) == 42

print(buffer)

assert buffer.pop() == 12
assert buffer.pop() == 9
assert buffer.pop() == 42

buffer.push(22)
buffer.push(33)
buffer.push(44)

print(buffer)

assert buffer.deque() == 22
assert len(buffer) == 2

assert buffer.deque() == 33

print(buffer)

buffer.push(55)

print(buffer)

assert buffer.pop() == 55
assert buffer.pop() == 44


<RingBuffer head=0 tail=1 len=1 ring: [5] buffer: [5, None, None] >
<RingBuffer head=0 tail=2 len=2 ring: [42, 9] buffer: [42, 9, None] >
<RingBuffer head=0 tail=0 len=3 ring: [42, 9, 12] buffer: [42, 9, 12] >
<RingBuffer head=0 tail=0 len=3 ring: [22, 33, 44] buffer: [22, 33, 44] >
<RingBuffer head=2 tail=0 len=1 ring: [44] buffer: [None, None, 44] >
<RingBuffer head=2 tail=1 len=2 ring: [44, 55] buffer: [55, None, 44] >
